## 讀入各種需要的套件

In [1]:
%env KERAS_BACKEND=tensorflow

env: KERAS_BACKEND=tensorflow


In [2]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist

from keras.utils import np_utils

from keras.layers import Dense, Activation, Flatten
from keras.layers import Conv2D, MaxPool2D
from keras.optimizers import SGD,Adam,Adadelta,Nadam

from keras.utils import to_categorical
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input
from keras.layers import concatenate ,add

Using TensorFlow backend.


## 準備訓練與驗證資料

In [3]:
#讀入手寫辨識資料集
(x_train, y_train), (x_test, y_test) = mnist.load_data()
#訓練資料reshape至可以進行預測的形狀
x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)
#1-hot encoding
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)
#normalization 老師:沒做會很飄
x_train = x_train/255
x_test = x_test/255

## 模型建立
建了四層filter分別為8,10,12,16的5*5卷積層
再接到10個neuron的dense層

In [4]:
#開一個函數學習機
model = Sequential()

In [5]:
model.add(Conv2D(8,(5,5),padding = 'same',input_shape = (28,28,1)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(10, (5,5), padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(12, (5,5), padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(16, (5,5), padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(10))
model.add(Activation('relu'))

model.add(Dense(10))
model.add(Activation('softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
model.compile(loss="categorical_crossentropy",optimizer=Nadam(lr=0.002),metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 8)         208       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 8)         0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 10)        2010      
_________________________________________________________________
activation_2 (Activation)    (None, 14, 14, 10)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 10)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 12)          3012      
__________

In [8]:
model.fit(x_train, y_train, batch_size=200, epochs=10, verbose=1, validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 19s 309us/step - loss: 0.0285 - acc: 0.9909 - val_loss: 0.0416 - val_acc: 0.9889
Epoch 2/10
60000/60000 [==============================] - 19s 311us/step - loss: 0.0258 - acc: 0.9917 - val_loss: 0.0400 - val_acc: 0.9883
Epoch 3/10
60000/60000 [==============================] - 19s 309us/step - loss: 0.0252 - acc: 0.9918 - val_loss: 0.0421 - val_acc: 0.9881
Epoch 4/10
60000/60000 [==============================] - 19s 310us/step - loss: 0.0233 - acc: 0.9926 - val_loss: 0.0397 - val_acc: 0.9891
Epoch 5/10
60000/60000 [==============================] - 21s 349us/step - loss: 0.0216 - acc: 0.9929 - val_loss: 0.0425 - val_acc: 0.9893
Epoch 6/10
60000/60000 [==============================] - 19s 323us/step - loss: 0.0201 - acc: 0.9935 - val_loss: 0.0415 - val_acc: 0.9889
Epoch 7/10
60000/60000 [==============================] - 20s 333us/step - loss: 0.0196 - acc: 0.9936 - val_loss: 0.

In [11]:
score = model.evaluate(x_test, y_test)
print(f'測試資料的 loss: {score[0]:.5f}')
print(f'測試資料的正確率: {score[1]}')

10000/10000 [==============================] - 2s 151us/step
測試資料的 loss: 0.05225
測試資料的正確率: 0.987


## 第二個模型

建了三層filter分別為4,6,8的5*5卷積層，再接到10個neuron的dense層，後面接的dense層使用sigmoid激發函數

In [9]:
mod = Sequential()

mod.add(Conv2D(4,(5,5),padding = 'same',input_shape = (28,28,1)))
mod.add(Activation('relu'))
mod.add(MaxPool2D(pool_size=(2,2)))

mod.add(Conv2D(6, (5,5), padding='same'))
mod.add(Activation('relu'))
mod.add(MaxPool2D(pool_size=(2,2)))

mod.add(Conv2D(8, (5,5), padding='same'))
mod.add(Activation('relu'))
mod.add(MaxPool2D(pool_size=(2,2)))

mod.add(Flatten())
mod.add(Dense(10))
mod.add(Activation('sigmoid'))

mod.add(Dense(10))
mod.add(Activation('softmax'))

mod.compile(loss="categorical_crossentropy",optimizer=Adam(lr=0.001),metrics=['accuracy'])
mod.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 28, 28, 4)         104       
_________________________________________________________________
activation_7 (Activation)    (None, 28, 28, 4)         0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 14, 14, 4)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 6)         606       
_________________________________________________________________
activation_8 (Activation)    (None, 14, 14, 6)         0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 7, 7, 6)           0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 7, 7, 8)           1208      
__________

In [10]:
mod.fit(x_train, y_train, batch_size=200, epochs=10, verbose=1, validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 32s 537us/step - loss: 1.6234 - acc: 0.6062 - val_loss: 1.0957 - val_acc: 0.8595
Epoch 2/10
60000/60000 [==============================] - 32s 533us/step - loss: 0.8580 - acc: 0.8863 - val_loss: 0.6643 - val_acc: 0.9120
Epoch 3/10
60000/60000 [==============================] - 34s 572us/step - loss: 0.5516 - acc: 0.9231 - val_loss: 0.4616 - val_acc: 0.9316
Epoch 4/10
60000/60000 [==============================] - 37s 623us/step - loss: 0.3994 - acc: 0.9374 - val_loss: 0.3488 - val_acc: 0.9441
Epoch 5/10
60000/60000 [==============================] - 39s 645us/step - loss: 0.3127 - acc: 0.9451 - val_loss: 0.2803 - val_acc: 0.9502
Epoch 6/10
60000/60000 [==============================] - 38s 634us/step - loss: 0.2563 - acc: 0.9517 - val_loss: 0.2271 - val_acc: 0.9574
Epoch 7/10
60000/60000 [==============================] - 36s 593us/step - loss: 0.2183 - acc: 0.9564 - val_loss: 0.

In [12]:
score1 = mod.evaluate(x_test, y_test)
print(f'測試資料的 loss: {score1[0]:.5f}')
print(f'測試資料的正確率: {score1[1]}')

10000/10000 [==============================] - 2s 154us/step
測試資料的 loss: 0.14669
測試資料的正確率: 0.9659


## 結果:兩個模型預測的結果分別為97.8%與96.5%的正確率